# Adau1761_0 IP 

This notebook serves as a quick demonstration of the audio codec being used in the **PYNQ-Z2 board**. A new IP has been introduced to make use of the codec. Before starting with this notebook please ensure you have the following: 
* Added the new audio.py file in the board
* Added the new pl.py file in the board
* Also, a new libsaudio.so is to be added

## How the new IP looks like?  

This is a screenshot of the addition done to the exsisting base overlay. Instead of the original audio IP block the new one looks like this
<p align="center">
<img src ="./sources/IP.JPG" width="100%" height="100%"/>
</p>


As we can see : 
* The **adau1761_0** IP is where the main AXI interactions take place. It also conists of a serializer, to serialize the audio going to the headphone jack, and a deserializer, to decode the sound coming from the MIC.
* The **axi_dma_0** IP is responsible for streaming audio data to the adau1761_0 through the _Slave AXI-Stream_ Interface of adau1761_0
* Thw **segement_stream_0** is responsible for controlling the _Master AXI_Stream_ Interface of adau1761_0

# Wavgen

This is a seprate python function to generate a sine wave and save it as a _.wav_ file. The function description is as follows: 
```
audio_write("name_of_the_file.wav", sampling rate, time period, frequency of sine wave)
```
( Make sure to keep this jupyter nb in the same place where the wavegen.py file is)

In [ ]:
from wavgen import audio_write

In [ ]:
audio_write("./output/samples.wav",100,5,44)

The waveform being generated:

In [ ]:
%matplotlib inline
import wave
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.fftpack import fft

wav_path = "./output/samples.wav"
with wave.open(wav_path, 'r') as wav_file:
    raw_frames = wav_file.readframes(-1)
    num_frames = wav_file.getnframes()
    num_channels = wav_file.getnchannels()
    sample_rate = wav_file.getframerate()
    sample_width = wav_file.getsampwidth()
    
temp_buffer = np.empty((num_frames, num_channels, 4), dtype=np.uint8)
raw_bytes = np.frombuffer(raw_frames, dtype=np.uint8)
temp_buffer[:, :, :sample_width] = raw_bytes.reshape(-1, num_channels, 
                                                    sample_width)
temp_buffer[:, :, sample_width:] = \
    (temp_buffer[:, :, sample_width-1:sample_width] >> 7) * 255
frames = temp_buffer.view('<i4').reshape(temp_buffer.shape[:-1])

In [ ]:
for channel_index in range(num_channels):
    plt.figure(num=None, figsize=(15, 3))
    plt.title('Audio in Time Domain (Channel {})'.format(channel_index))
    plt.xlabel('Time in s')
    plt.ylabel('Amplitude')
    time_axis = np.arange(0, num_frames/sample_rate, 1/sample_rate)
    plt.plot(time_axis, frames[:, channel_index])
    plt.show()

# Initialization

### Create a new audio object

In [ ]:
from audio import *
base=Overlay("./sources/AXIS_audio.bit")
Audiobj=base.adau1761_0

## Bypass audio
Users can select either `LINE_IN`, or `HP+MIC` as the input port.
In the following example, we choose `LINE_IN`. To choose `MIC`:
```python
pAudio.select_microphone()
```
or choose `LINE_IN`:
```python
pAudio.select_line_in()
```

In [ ]:
Audiobj.select_microphone()

## Load and play
Load a sample and play the loaded sample.

In [ ]:
Audiobj.load("./sources/sine.wav")

## Play function 

## Stream 

Copy the list genrated from the audio file (the load() function generates this) into an array. 

In [ ]:
buf = Audiobj.buffer

Create a continous allocated memory numpy array 

In [ ]:
import pynq.lib.dma
from pynq import Xlnk

xlnk = Xlnk()

dma_send = base.axi_dma_0
cma_ar = xlnk.cma_array(buf.shape, buf.dtype)
cma_ar[:] = buf

The `playinit()` initializes the various audio codec registers.

The numpy array which we declared above is passed onto the **DMA** send channel.

In [ ]:
async def play_audio():
    Audiobj.playinit()
    dma_send.sendchannel.transfer(cma_ar)
    await dma_send.sendchannel.wait_async()

## Monitoring the CPU Usage

To see how CPU usages is impacted by the audio stream we create another task that prints out the current CPU utilisation every 3 seconds.

In [ ]:
import psutil
import asyncio

@asyncio.coroutine
def print_cpu_usage():
    # Calculate the CPU utilisation by the amount of idle time
    # each CPU has had in three second intervals
    last_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
    while True:
        yield from asyncio.sleep(3)
        next_idle = [c.idle for c in psutil.cpu_times(percpu=True)]
        usage = [(1-(c2-c1)/3) * 100 for c1,c2 in zip(last_idle, next_idle)]
        print("CPU Usage: {0:3.2f}%, {1:3.2f}%".format(*usage))
        last_idle = next_idle



In [ ]:
audio_task = asyncio.ensure_future(play_audio())
cpu_task = asyncio.ensure_future(print_cpu_usage())
asyncio.get_event_loop().run_until_complete(audio_task)

The `playend()` mutes the various audio codec registers which were being used.

In [ ]:
Audiobj.playend()

### Slave

The play() function of the AXI-Slave is not configured properly. Please note. 

In [ ]:
Audiobj.play()

## Record function

Records a 5-second sample and is stored in a continous memory allocated array : 

### Stream 

Enter the time for which the recording will take place:

In [ ]:
seconds = 5

Create a continous allocated memory numpy array 

In [ ]:
import numpy as np
import pynq.lib.dma
from pynq import Xlnk

xlnk = Xlnk()

dma_send = base.axi_dma_0
cma_ar = xlnk.cma_array(shape = seconds * 2 * 48000, dtype = "uint32")

The segement_stream is responsible for managing the AXI-Stream transactions between the `MIC` (Master AXI Stream) of the audio codec and the PS (Slave Stream).

In [ ]:
base.segment_stream_0.write(0, seconds * 2 * 48000)

After this we have to send the audio array to the DMA

In [ ]:
Audiobj.recordinit(seconds)
dma_send.recvchannel.transfer(cma_ar)
dma_send.recvchannel.wait()

And then to play it, we will use the DMA again to play from the array:

In [ ]:
Audiobj.playinit()
dma_send.sendchannel.transfer(cma_ar)
dma_send.sendchannel.wait()

In [ ]:
Audiobj.playend()

### Slave

This here again is the recording function, but uses the **AXI-Slave** instead of the **AXI-Stream**.

In [ ]:
Audiobj.record(seconds=5)

In [ ]:
Audiobj.play()